## Stage Out (Ancillary) Example

Steps
- Install Data Service (DS) Client Library
- Set Log Level
- Set Environment variables for Stage-Out (Ancillary
- Try Stage-Out Ancillary Files
- Check the results

In [1]:
from anyio.streams import file
%pip install mdps-ds-lib

Note: you may need to restart the kernel to use updated packages.


### Setting the Log Level
- Log level mappings:
- 10 = debug
- 20 = info
- 30 = warning
- 40 = error

In [2]:
import logging
log_level = 30
logging.basicConfig(level=log_level, format="%(asctime)s [%(levelname)s] [%(name)s::%(lineno)d] %(message)s")

### Stage-out (Ancillary) Only Environment Variables

Follow this link for more information: https://app.gitbook.com/o/xZRqGQeQXJ0RP4VMj7Lq/s/UMIRhLdbRQTvMWop8Il9/developer-docs/data/docs/users-guide/stage-out

In [3]:
from mdps_ds_lib.lib.utils.file_utils import FileUtils
import os

os.environ['AWS_ACCESS_KEY_ID'] = 'xxx'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'xxx'
os.environ['AWS_SESSION_TOKEN'] = 'xxx'

os.environ['GRANULES_UPLOAD_TYPE'] = 'UPLOAD_AUXILIARY_FILE_AS_GRANULE'  # Setting uploading as auxiliary
os.environ['STAGING_BUCKET'] = 'uds-sbx-cumulus-staging'  # S3 bucket where they will reside
os.environ['VERIFY_SSL'] = 'FALSE'  # Optional param.
os.environ['BASE_DIRECTORY'] = 'auxiliary_files'  # Base folder to upload

os.environ['OUTPUT_DIRECTORY'] = 'auxiliary_files/output_dir'  # the success / failure results to be stored locally for rewview
os.environ['OUTPUT_FILE'] = 'auxiliary_files/output_dir/stage_out_result.json'  # file path where the result overview is written locally for

tenant = 'UDS_DEMO'
tenant_venue = 'TEST'
collection_name = 'UDS_UNIT_COLLECTION'
collection_version = '24.09.10.11.00'.replace('.', '')
temp_collection_id = f'URN:NASA:UNITY:{tenant}:{tenant_venue}:{collection_name}___{collection_version}'
os.environ['COLLECTION_ID'] = temp_collection_id  # Setting Collection ID.
# Note that Collection ID needs to follow the standard.

FileUtils.mk_dir_p(os.environ.get('OUTPUT_DIRECTORY'))


#### Mocking some ancillary files

In [4]:
granules_dir = os.path.join('auxiliary_files')
FileUtils.mk_dir_p(granules_dir)  # base directory
total_files = 3  # Uploading 3 mock files

for i in range(1, total_files+1):
    filename = f'mocked_general_file{i:02d}'
    with open(os.path.join(granules_dir, f'{filename}'), 'w') as ff:  # Creating Data File
        ff.write('mocked_general_file')

#### Performing actual stage-out
- Note that there is no dry-run on ancillary stage-out since there are no settings that client setup.

In [5]:
from glob import glob

# Cleaning old result files.

old_result_files = glob(os.path.join(os.environ['OUTPUT_DIRECTORY'], '*'))
for each_file in old_result_files:
    FileUtils.remove_if_exists(each_file)

In [6]:

from mdps_ds_lib.stage_in_out.upoad_granules_factory import UploadGranulesFactory
from mdps_ds_lib.stage_in_out.stage_in_out_utils import StageInOutUtils

# Hardcoded method call. All params are set via environment previously
upload_result_str = UploadGranulesFactory().get_class(os.getenv('GRANULES_UPLOAD_TYPE', UploadGranulesFactory.UPLOAD_S3_BY_STAC_CATALOG)).upload()
StageInOutUtils.write_output_to_file(upload_result_str)
print('done')

2025-02-11 11:57:55,489 [AUDIT] [mdps_ds_lib.stage_in_out.upload_arbitrary_files_as_granules::11] uploading auxiliary file: auxiliary_files/mocked_general_file02
2025-02-11 11:57:55,489 [AUDIT] [mdps_ds_lib.stage_in_out.upload_arbitrary_files_as_granules::11] uploading auxiliary file: auxiliary_files/mocked_general_file03
2025-02-11 11:57:55,489 [AUDIT] [mdps_ds_lib.stage_in_out.upload_arbitrary_files_as_granules::11] uploading auxiliary file: auxiliary_files/mocked_general_file01


done
